# PyDipAPI - Datenvisualisierung Tutorial

Dieses Notebook zeigt, wie Sie Bundestag-Daten visualisieren.

## Inhalt:
1. **Matplotlib** - Statische Diagramme
2. **Datenanalyse** - Parteien und Statistiken
3. **Zeitreihen-Analysen** - Trends ueber Zeit
4. **Export-Funktionen** - Diagramme und Daten speichern
5. **Dashboard-Konzepte** - Uebersichtsdarstellungen

In [ ]:
import sys
import os
from collections import Counter
from datetime import datetime, timedelta
sys.path.insert(0, os.path.abspath('..'))

from pydipapi import DipAnfrage

try:
    import matplotlib.pyplot as plt
    print("Matplotlib verfuegbar")
except ImportError:
    print("Installiere matplotlib...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "matplotlib"])
    import matplotlib.pyplot as plt

try:
    import pandas as pd
    print("Pandas verfuegbar")
except ImportError:
    print("Installiere pandas...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas"])
    import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

API_KEY = "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN"

if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    dip = DipAnfrage(api_key=API_KEY)
    print("Datenvisualisierung Tutorial bereit!")
else:
    print("BITTE ERSETZEN SIE DEN PLATZHALTER MIT IHREM ECHTEN API-SCHLUESSEL!")
    print("Den API-Schluessel erhalten Sie unter: https://dip.bundestag.de/api/")

## Daten laden

In [ ]:
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    print("Lade Daten...")
    
    try:
        persons_data = dip.get_person(anzahl=50)
        print(f"{len(persons_data)} Personen geladen")
        
        documents_data = dip.get_drucksache(anzahl=30)
        print(f"{len(documents_data)} Dokumente geladen")
        
        activities_data = dip.get_aktivitaet(anzahl=30)
        print(f"{len(activities_data)} Aktivitaeten geladen")
    except Exception as e:
        print(f"Fehler: {e}")
        persons_data = []
        documents_data = []
        activities_data = []
else:
    print("Bitte API-Schluessel konfigurieren.")
    persons_data = []
    documents_data = []
    activities_data = []

## Parteienverteilung visualisieren

In [ ]:
if persons_data:
    # Parteien extrahieren
    parties = []
    for person in persons_data:
        party = (person.get('fraktion') or 
                person.get('partei') or 
                'Unbekannt')
        parties.append(party)
    
    party_counts = Counter(parties)
    
    print("Parteienverteilung:")
    for party, count in party_counts.most_common():
        print(f"  {party}: {count} Personen")
    
    # Balkendiagramm
    plt.figure(figsize=(12, 6))
    parties_list = list(party_counts.keys())
    counts_list = list(party_counts.values())
    
    bars = plt.bar(parties_list, counts_list, color='steelblue')
    plt.title('Parteienverteilung', fontsize=16, fontweight='bold')
    plt.xlabel('Partei/Fraktion')
    plt.ylabel('Anzahl Personen')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    
    # Werte auf Balken
    for bar, count in zip(bars, counts_list):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                str(count), ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('parteien_verteilung.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("Diagramm gespeichert als 'parteien_verteilung.png'")
else:
    print("Keine Personendaten verfuegbar")

## Zeitreihen-Analyse: Dokumente ueber Zeit

In [ ]:
if documents_data:
    # Datum extrahieren und gruppieren
    dates = []
    for doc in documents_data:
        date_str = doc.get('datum', '')
        if date_str:
            try:
                # Datum parsen (Format kann variieren)
                if 'T' in date_str:
                    date_str = date_str.split('T')[0]
                date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                dates.append(date_obj)
            except:
                pass
    
    if dates:
        # Nach Monaten gruppieren
        months = [d.strftime('%Y-%m') for d in dates]
        month_counts = Counter(months)
        
        # Sortieren
        sorted_months = sorted(month_counts.items())
        months_list = [m[0] for m in sorted_months]
        counts_list = [m[1] for m in sorted_months]
        
        # Zeitreihen-Diagramm
        plt.figure(figsize=(12, 6))
        plt.plot(months_list, counts_list, marker='o', linewidth=2, markersize=8)
        plt.title('Dokumente ueber Zeit', fontsize=16, fontweight='bold')
        plt.xlabel('Monat')
        plt.ylabel('Anzahl Dokumente')
        plt.xticks(rotation=45, ha='right')
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.savefig('dokumente_zeitreihe.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("Diagramm gespeichert als 'dokumente_zeitreihe.png'")
    else:
        print("Keine Datumsinformationen in Dokumenten gefunden")
else:
    print("Keine Dokumentdaten verfuegbar")

## Aktivitaetstypen-Verteilung

In [ ]:
if activities_data:
    # Aktivitaetstypen extrahieren
    activity_types = [a.get('aktivitaetstyp', 'Unbekannt') for a in activities_data]
    type_counts = Counter(activity_types)
    
    print("Aktivitaetstypen:")
    for activity_type, count in type_counts.most_common():
        print(f"  {activity_type}: {count}")
    
    # Kreisdiagramm
    plt.figure(figsize=(10, 8))
    types_list = list(type_counts.keys())
    counts_list = list(type_counts.values())
    
    plt.pie(counts_list, labels=types_list, autopct='%1.1f%%', startangle=90)
    plt.title('Verteilung der Aktivitaetstypen', fontsize=16, fontweight='bold')
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig('aktivitaetstypen_verteilung.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("Diagramm gespeichert als 'aktivitaetstypen_verteilung.png'")
else:
    print("Keine Aktivitaetsdaten verfuegbar")

## Datenexport: CSV

In [ ]:
if persons_data:
    # Personen-Daten in DataFrame konvertieren
    persons_df = pd.DataFrame(persons_data)
    
    # Wichtige Spalten auswaehlen
    important_cols = ['id', 'vorname', 'nachname', 'fraktion', 'partei']
    available_cols = [col for col in important_cols if col in persons_df.columns]
    
    if available_cols:
        persons_export = persons_df[available_cols]
        persons_export.to_csv('personen_export.csv', index=False)
        print(f"Personendaten exportiert: {len(persons_export)} Zeilen")
        print(f"Spalten: {', '.join(available_cols)}")
        print("Datei: 'personen_export.csv'")
    else:
        print("Keine verfuegbaren Spalten zum Exportieren")
else:
    print("Keine Personendaten verfuegbar")

## Datenexport: JSON

In [ ]:
import json

if documents_data:
    # Dokumente in JSON exportieren
    # Nur wichtige Felder auswaehlen
    export_data = []
    for doc in documents_data[:10]:  # Erste 10 Dokumente
        export_doc = {
            'id': doc.get('id'),
            'titel': doc.get('titel'),
            'drucksachetyp': doc.get('drucksachetyp'),
            'datum': doc.get('datum')
        }
        export_data.append(export_doc)
    
    with open('dokumente_export.json', 'w', encoding='utf-8') as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)
    
    print(f"Dokumentendaten exportiert: {len(export_data)} Dokumente")
    print("Datei: 'dokumente_export.json'")
else:
    print("Keine Dokumentdaten verfuegbar")

## Dashboard-Konzept: Kombinierte Visualisierung

In [ ]:
if persons_data and documents_data:
    # Erstelle ein Dashboard mit mehreren Subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Bundestag-Daten Dashboard', fontsize=18, fontweight='bold')
    
    # 1. Parteienverteilung
    parties = [p.get('fraktion') or p.get('partei') or 'Unbekannt' for p in persons_data]
    party_counts = Counter(parties)
    
    axes[0, 0].bar(party_counts.keys(), party_counts.values(), color='steelblue')
    axes[0, 0].set_title('Parteienverteilung')
    axes[0, 0].set_ylabel('Anzahl')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Dokumenttypen
    doc_types = [d.get('drucksachetyp', 'Unbekannt') for d in documents_data]
    type_counts = Counter(doc_types)
    
    axes[0, 1].bar(type_counts.keys(), type_counts.values(), color='coral')
    axes[0, 1].set_title('Dokumenttypen')
    axes[0, 1].set_ylabel('Anzahl')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Aktivitaetstypen (wenn verfuegbar)
    if activities_data:
        act_types = [a.get('aktivitaetstyp', 'Unbekannt') for a in activities_data]
        act_counts = Counter(act_types)
        
        axes[1, 0].bar(act_counts.keys(), act_counts.values(), color='lightgreen')
        axes[1, 0].set_title('Aktivitaetstypen')
        axes[1, 0].set_ylabel('Anzahl')
        axes[1, 0].tick_params(axis='x', rotation=45)
    else:
        axes[1, 0].text(0.5, 0.5, 'Keine Aktivitaetsdaten', 
                       ha='center', va='center', transform=axes[1, 0].transAxes)
        axes[1, 0].set_title('Aktivitaetstypen')
    
    # 4. Statistik-Text
    stats_text = f"\nStatistiken:\n"
    stats_text += f"Personen: {len(persons_data)}\n"
    stats_text += f"Dokumente: {len(documents_data)}\n"
    if activities_data:
        stats_text += f"Aktivitaeten: {len(activities_data)}"
    
    axes[1, 1].text(0.5, 0.5, stats_text, ha='center', va='center', 
                   transform=axes[1, 1].transAxes, fontsize=14)
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig('dashboard.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("Dashboard gespeichert als 'dashboard.png'")
else:
    print("Nicht genug Daten fuer Dashboard verfuegbar")

## Zusammenfassung

In diesem Notebook haben Sie gelernt:

- **Matplotlib**: Statische, hochwertige Diagramme erstellen
- **Datenanalyse**: Parteienverteilung, Aktivitaetstypen, Dokumenttypen
- **Zeitreihen**: Trends ueber Zeit visualisieren
- **Datenexport**: CSV und JSON Export
- **Dashboard**: Kombinierte Visualisierungen

### Erstellte Dateien:
- `parteien_verteilung.png` - Parteienverteilungs-Diagramm
- `dokumente_zeitreihe.png` - Zeitreihen-Diagramm
- `aktivitaetstypen_verteilung.png` - Aktivitaetstypen-Verteilung
- `dashboard.png` - Kombiniertes Dashboard
- `personen_export.csv` - Personendaten als CSV
- `dokumente_export.json` - Dokumentendaten als JSON

### Weitere Moeglichkeiten:
- Interaktive Visualisierungen mit Plotly
- Erweiterte statistische Analysen mit Pandas
- HTML-Reports generieren
- Automatisierte Berichte

**Alle PyDipAPI Tutorials erfolgreich abgeschlossen!**